<a href="https://colab.research.google.com/github/dosanchez/data-science-learning/blob/main/exercises_chpt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix #convert pandas df to scipy spase matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/datasets/titanic.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df[df["Age"].isna()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q


In [8]:
import os
import chardet
import re
from sklearn.base import BaseEstimator, TransformerMixin

class MailFileImporter (BaseEstimator, TransformerMixin):

    def __init__(self, include_headers = False):
      self.include_headers = include_headers

    def fit(self, directory = ".", spam = False):
      return self

    def transform(self, directory = ".", spam = False):
      directory_path = directory

      #initializing values
      mail = []
      record_word_col = np.zeros((1,2))

      # Iterate through all files in the directory

      for filename in sorted(os.listdir(directory_path)):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
          # Check if it's a file (not a directory)
        if os.path.isfile(file_path):
            with open(file_path, 'rb') as raw_file:
              raw_data = raw_file.read()
              guessed_encoding = chardet.detect(raw_data)['encoding']

            with open(file_path, 'r', encoding=guessed_encoding) as file:
              content = file.read()

            pattern = re.compile(r"\n[A-Z]\S*:") #s[^A-Z]\S*:
            matches = pattern.finditer(content)

            body_pos = 0

            for i in matches:
              if i.span()[1]>body_pos:
                body_pos = i.span()[1]

            body_words = content[body_pos:].split()
            print(len(body_words))
            record_to_add = np.array(body_words)

            mail_type = np.array([spam for i in range(len(body_words))], dtype = np.bool)#.reshape(len(body_words),1)
            record_to_add = np.column_stack([mail_type, record_to_add])
            record_word_col = np.vstack([record_word_col, record_to_add])

      print(np.shape(record_word_col))
      print(type(record_word_col))
      print(record_word_col)
            #print (mail_type)
            #print(content[body_pos:])

            #header = content[:body_pos]
      return record_word_col

In [9]:
mails = MailFileImporter().transform("/content/drive/MyDrive/datasets/spam")

(1, 2)
<class 'numpy.ndarray'>
[[0. 0.]]


In [10]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
mails_1_hot = cat_encoder.fit_transform(mails[:,1].reshape(-1,1))

In [11]:
print(cat_encoder.categories_[0])
#print("All" in cat_encoder.categories_)


[0.]


In [12]:
mails_1_hot.shape

(1, 1)

In [13]:
type(mails_1_hot)

scipy.sparse._csr.csr_matrix

In [14]:
dfprueba= df.iloc[:10]

In [15]:
dfprueba.iloc[:, [0, 1, 2, 3, 4, 5, 9]]

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,8.0500
5,6,0,3,"Moran, Mr. James",male,NaN,8.4583
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,51.8625
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,21.0750
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,11.1333
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,30.0708


In [16]:
prueba_cat_encoder = OneHotEncoder()
prueba_one_hot = prueba_cat_encoder.fit_transform(dfprueba.iloc[:, [3,4]])

In [17]:
path1 = "/content/drive/MyDrive/datasets/spam"
path2 ='spam'
print(os.path.join(path1, path2))

/content/drive/MyDrive/datasets/spam/spam


In [18]:
print(prueba_cat_encoder.categories_)

[array(['Allen, Mr. William Henry', 'Braund, Mr. Owen Harris',
       'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
       'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
       'Heikkinen, Miss. Laina',
       'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
       'McCarthy, Mr. Timothy J', 'Moran, Mr. James',
       'Nasser, Mrs. Nicholas (Adele Achem)',
       'Palsson, Master. Gosta Leonard'], dtype=object), array(['female', 'male'], dtype=object)]


In [19]:
print(np.full(10, "spam"))

['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam']


In [20]:
class MailFileImporter1 (BaseEstimator, TransformerMixin):

    def __init__(self, include_headers = False):
      self.include_headers = include_headers

    def fit(self, directory = ".", spam = False):
      return self

    def transform(self, directory = "."):
      directory_path = directory

      #initializing values
      mail = []
      record = np.zeros((1,3))
      record_no = 0
      record_word_col = np.zeros((1,3))

      # Iterate through all files in the directory & makes word matrix
      for pocket in ["ham", "spam"]:
        file_path = os.path.join(directory_path, pocket)
        for filename in sorted(os.listdir(file_path)):
          ## Construct the full file path
          file_path = os.path.join(directory_path, pocket)#resets filepath 2 dir
          file_path = os.path.join(file_path, filename)
            ## Check if it's a file (not a directory)
          if os.path.isfile(file_path):
            record_no += 1
            print(record_no)
            with open(file_path, 'rb') as raw_file:
              raw_data = raw_file.read()
              guessed_encoding = chardet.detect(raw_data)['encoding']

            with open(file_path, 'r', encoding=guessed_encoding) as file:
              content = file.read()

            pattern = re.compile(r"\n[A-Z]\S*:") #s[^A-Z]\S*:
            matches = pattern.finditer(content)

            body_pos = 0

            for i in matches:
              if i.span()[1]>body_pos:
                body_pos = i.span()[1]

            body_words = content[body_pos:].split()
            record_to_add = np.array(body_words)
            record_no_to_add = np.array([record_no for i in range(len(body_words))])

            #classifies reacord as spam (1) or ham (0) before one hot
            if pocket == "ham":
              mail_type = np.array([0 for i in range(len(body_words))])
            else:
              mail_type = np.array([1 for i in range(len(body_words))])

            record_to_add = np.column_stack((record_no_to_add, mail_type, record_to_add))
            record = np.vstack([record, record_to_add])
            record = np.delete(record, 0, axis=0)

      #one hot encoding mail body
      mail_encoder = OneHotEncoder()
      mail_one_hot = mail_encoder.fit_transform(record[:,2].reshape(-1,1))
      #print(mail_encoder.categories_[0])
      dataset_one_hot = np.hstack((record[:,:2], mail_one_hot.toarray()))
      #pandas df sum up
      columns = ['record_no', 'spam'] + mail_encoder.categories_[0].tolist()
      df = pd.DataFrame(dataset_one_hot, columns = columns, dtype="float32")
      df_sum = df.groupby(['record_no', 'spam']).sum()
      df_sum.reset_index(inplace=True)
      df_sum.drop('record_no', axis=1, inplace=True)
      df_sum.reset_index(drop=True)
      #normalize all rows but pocket and make it X matrix
      df_sum_no_pocket = df_sum.drop(columns=['spam'])
      X = normalize(df_sum_no_pocket, norm='l2')
      X = pd.DataFrame(X, columns=df_sum_no_pocket.columns)
      #preparing y
      y = df_sum['spam']
      return X,y

In [21]:
X,y = MailFileImporter1().transform("/content/drive/MyDrive/datasets/spam")


1
2
3
4
5
6
7
8
9
10


In [24]:
print(type(y))
print(y.shape)
print(y.head())


<class 'pandas.core.series.Series'>
(10,)
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: spam, dtype: float32


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Binary classifier

In [33]:
sgd_cls = SGDClassifier(random_state=42)
sgd_cls.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [30]:
print(Xtrain_spam.shape)

(3, 1024)
